## 写在前面

- LangChain 也是一套面向大模型的开发框架（SDK）
- LangChain 并不完美，还在不断迭代中
- 学习 Langchain 更重要的是借鉴其思想，具体的接口和模块可能很快就会改变


## LangChain vs. Semantic Kernel

[![Star History Chart](https://api.star-history.com/svg?repos=langchain-ai/langchain,microsoft/semantic-kernel&type=Date)](https://star-history.com/#langchain-ai/langchain&microsoft/semantic-kernel&Date)


LangChain 完胜？我们接下来仔细看看。


| 功能/工具           | LangChain                       | Semantic Kernel                  |
|-------------------|:---------------------------------:|:----------------------------------:|
| 版本号        |  0.1.0  | python-0.4.4.dev  |
| 适配的 LLM        | 多   | 少 + 外部生态   |
| Prompt 工具        | 支持    | 支持     |
| Prompt 函数嵌套    | 需要通过 LCEL | 支持        |
| Prompt 模板嵌套    | 不支持  | 不支持       |
| 输出解析工具       | 支持  | 不支持  |
| 上下文管理工具           | 支持 | C#版支持，Python版尚未支持  |
| 内置工具           | 多，但良莠不齐  | 少 + 外部生态  |
| 三方向量数据库适配           | 多 | 少 + 外部生态  |
| 服务部署 | LangServe | 与 Azure 衔接更丝滑
| 管理工具 | LangSmith/LangFuse | Prompt Flow

## LangChain 的核心组件

<img src="langchain_stack.png" style="margin-left: 0px" width=1000px>

1. 模型 I/O 封装
   - LLMs：大语言模型
   - Chat Models：一般基于 LLMs，但按对话结构重新封装
   - PromptTemple：提示词模板
   - OutputParser：解析输出
2. 数据连接封装
   - Document Loaders：各种格式文件的加载器
   - Document Transformers：对文档的常用操作，如：split, filter, translate, extract metadata, etc
   - Text Embedding Models：文本向量化表示，用于检索等操作（啥意思？别急，后面详细讲）
   - Verctorstores: （面向检索的）向量的存储
   - Retrievers: 向量的检索
3. 记忆封装
   - Memory：这里不是物理内存，从文本的角度，可以理解为“上文”、“历史记录”或者说“记忆力”的管理
4. 架构封装

   - Chain：实现一个功能或者一系列顺序功能组合

   - Agent：根据用户输入，自动规划执行步骤，自动选择每步需要的工具，最终完成用户指定的功能
     - Tools：调用外部功能的函数，例如：调 google 搜索、文件 I/O、Linux Shell 等等
     - Toolkits：操作某软件的一组工具集，例如：操作 DB、操作 Gmail 等等

   - Chain及Agent架构设计（下图为ReAct类型智能体，后文会介绍）
   
   <img src="langchain.png" style="margin-left: 0px" width=600px>

## 一、模型 I/O 封装

> 官方文档：https://python.langchain.com/v0.1/docs/modules/model_io/


把不同的模型，统一封装成一个接口，方便更换模型而不用重构代码。


<img src="model_io.jpg" style="margin-left: 0px" width=800px>

在Model I/O这一流程中，LangChain抽象的组件主要有三个：

- Language models
- Prompts
- Output parsers



安装依赖

In [ ]:
#安装最新版本
!pip install langchain==0.2.13
!pip install langchain-openai
!pip install langchain_community

### 1.1 模型封装：LLM vs. ChatModel

LangChain中提供了三类模型的封装，LLM、ChatModel和Embedding，下面聊一下LLM和ChatModel的区别：

（1）LLM：输入和输出都是**纯文本**的模型（text in， text out）

In [15]:
from langchain_openai import OpenAI
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

text = "你是谁？"

llm = OpenAI()

llm.invoke(text)

'\n我是一个人工智能助手，被设计来回答问题、提供帮助和进行对话。'

（2）ChatModel：是LLM的变体，对聊天场景进行了抽象。输入不是纯文本，而是chat message列表，输出也是chat message。

    chat message = text + 消息类型(System, Human, AI)

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

chat_model = ChatOpenAI()

messages = [HumanMessage(content=text)]

chat_model.invoke(messages)


AIMessage(content='你是张三。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 26, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-25c56638-f0f4-4a8c-af2e-62149d92712f-0', usage_metadata={'input_tokens': 26, 'output_tokens': 5, 'total_tokens': 31})

- chat message目前一共有5种类型，每种message至少包含角色（role）和内容（content）两个参数：

    - HumanMessage：等价于OpenAI接口中的user role

    - AIMessage：等价于OpenAI接口中的assistant role
    
    - SystemMessage：等价于OpenAI接口中的system role

    - FunctionMessage：function call的结果。除了role和content之外，它还有一个name参数，表示函数的名称。
    
    - ToolMessage：tool call的结果。除了role和content之外，它还有一个tool_call_id参数。

    详细文档：https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/

In [24]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

messages = [
    SystemMessage(content="你是AGIClass的课程助理。"), 
    HumanMessage(content="我是学员，我叫张三"),
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁？")
]

chat_model.invoke(messages)


AIMessage(content='您是张三，一位学员。您可以告诉我您有什么问题或者需要帮助的地方。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 51, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bcb315c1-bf88-473f-b855-182c3bfe5b15-0', usage_metadata={'input_tokens': 51, 'output_tokens': 33, 'total_tokens': 84})


（3）**为什么要区分LLM和ChatModel？**

- LLM主要处理一次性问题，适用于回答问题、生成文本等场景
- Chat Model 则专注于多轮对话场景，不仅接受用户的输入，还考虑对话的上下文


（4）通过模型封装，可以实现不同模型的统一接口调用

In [ ]:

# 从langchain_community导入百度千帆大模型
from langchain_community.chat_models import ErnieBotChat
from langchain_core.messages import HumanMessage

ernie = ErnieBotChat()

messages = [
    HumanMessage(content="你是谁") 
]

ernie.invoke(messages)

### 1.2 Prompt模板封装

> 官方文档：https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/

#### 1.2.1 PromptTemplate
- PromptTemplate是纯字符串形式
- 使用类似于str.format()语法进行参数补全

In [37]:
from langchain_core.prompts import PromptTemplate
# 参数用{}包裹
template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print(template.input_variables)
print(template.format(subject='小明'))

['subject']
给我讲个关于小明的笑话


#### 1.2.2 ChatPromptTemplate
- ChatPromptTemplate是对chat message的封装，携带角色信息

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        # 等价于 {"role": "system", "content": "你是{product}的客服助手。你的名字叫{name}"},
        SystemMessagePromptTemplate.from_template("你是{product}的客服助手。你的名字叫{name}"),
        # 等价于 {"role": "user", "content": "{query}"},
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

prompt = template.format_messages(
        product="AGI课堂",
        name="瓜瓜",
        query="你是谁"
    )
print(prompt)


[SystemMessage(content='你是AGI课堂的客服助手。你的名字叫瓜瓜'), HumanMessage(content='你是谁')]


#### 1.2.3 从文件加载Prompt模板

1. Yaml格式

``` yml
 _type: prompt
input_variables:
    ["adjective", "content"]
template: 
    Tell me a {adjective} joke about {content}.
```

2. JSON格式

```JSON

{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}
```

3. Template单独存放

- 注意点：使用`template_path`
- 从代码文件的相对路径找模板文件


```JSON
{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template_path": "simple_template.txt"
}
```

加载方式

In [3]:
from langchain_core.prompts import load_prompt

# prompt = load_prompt("simple_prompt.yaml")
prompt = load_prompt("simple_prompt.json")

print(prompt.format(adjective="funny", content="fox"))

Tell me a funny joke about fox.


### 1.3 输出封装 OutputParser

自动把 LLM 输出的字符串转换为指定格式

LangChain 内置的 OutputParser 包括:

- PydanticParser
- ListParser
- DatetimeParser
- EnumParser
- XMLParser

等等， 官方文档：https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/

### 1.3.1 Pydantic (JSON) Parser

- Pydantic 是一个用于数据验证和数据解析的 Python 库。
- PydanticParser可以自动根据Pydantic类的定义，**生成JSON格式的输出**。
- Pydantic v1版本即将弃用，建议使用v2版本：https://python.langchain.com/v0.2/docs/how_to/pydantic_compatibility/
- 使用PydanticParser的好处，不需要在prompt中描述一大串json格式的定义

In [1]:
from pydantic import BaseModel, Field
from typing import List, Dict

# 定义你的输出对象
class Date(BaseModel):
    year: int = Field(description="Year")
    month: int = Field(description="Month")
    day: int = Field(description="Day")
    era: str = Field(description="BC or AD")

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# 根据Pydantic对象的定义，构造一个OutputParser
parser = PydanticOutputParser(pydantic_object=Date)

template = """提取用户输入中的日期。
{format_instructions}
用户输入:
{query}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    # 预先赋值：直接从OutputParser中获取输出描述，并对模板的变量预先赋值
    # https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/
    partial_variables={"format_instructions": parser.get_format_instructions()} 
)

print("====Format Instruction=====")
print(parser.get_format_instructions())


query = "2023年四月6日天气晴..."
model_input = prompt.format_prompt(query=query)

print("====Prompt=====")
print(model_input)

output = model.invoke(model_input)
print("====Output=====")
print(output)
print("====Parsed=====")
date = parser.parse(output.content)
print(date)

====Format Instruction=====
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"year": {"description": "Year", "title": "Year", "type": "integer"}, "month": {"description": "Month", "title": "Month", "type": "integer"}, "day": {"description": "Day", "title": "Day", "type": "integer"}, "era": {"description": "BC or AD", "title": "Era", "type": "string"}}, "required": ["year", "month", "day", "era"]}
```
====Prompt=====
text='提取用户输入中的日期。\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo

### 1.3.2 Auto-Fixing Parser

利用LLM自动根据解析异常修复并重新解析

In [8]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model="gpt-3.5-turbo"))

#我们把之前output的格式改错
output = output.content.replace("4","四")
print("===格式错误的Output===")
print(output)
try:
    date = parser.parse(output)
except Exception as e:
    print("===出现异常===")
    print(e)
    
#用OutputFixingParser自动修复并解析
date = new_parser.parse(output)
print("===重新解析结果===")
print(date)

===格式错误的Output===
{
  "year": 2023,
  "month": 四,
  "day": 6,
  "era": "AD"
}
===出现异常===
Invalid json output: {
  "year": 2023,
  "month": 四,
  "day": 6,
  "era": "AD"
}
===重新解析结果===
year=2023 month=4 day=6 era='AD'


猜一下OutputFixingParser是怎么做到的？ 再调一遍大模型告诉他哪里不对，让它修改

### 1.4、小结

1. LangChain 统一封装了各种模型的调用接口
2. LangChain 提供了提示词模板类，可以自定义带变量的模板
3. LangChain 提供了一些输出解析器，用于将大模型的输出解析成结构化对象；额外带有自动修复功能。
4. 上述模型属于 LangChain 中较为优秀的部分；美中不足的是 OutputParser 自身的 Prompt 维护在代码中，耦合度较高。

## 二、数据连接封装

<img src="data_connection.jpg" style="margin-left: 0px" width=600px>

### 2.1 文档加载器：Document Loaders

> 官方文档：https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/

In [ ]:
# 加载PDF需要的
!pip install pypdf
# 加载docx需要的
!pip install python-docx
!pip install docx2txt

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("体检中心问答.docx")
doc = loader.load()
content = doc[0].page_content

In [ ]:
content[0:500]

### 2.2 文档处理器 TextSplitter

> 官方文档：https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n\n\n"],
    chunk_size=0, # chunk_size=0, 它会严格按照separators进行分割
    chunk_overlap=0,  # 思考：为什么要做overlap
    length_function=len
)

paragraphs = text_splitter.create_documents([content])
for para in paragraphs:
    print(para.page_content.strip())
    print('-------')

<div class="alert alert-danger">
LangChain 的 PDFLoader 和 TextSplitter 实现都比较粗糙，实际生产中不建议使用。
</div>

### 2.3、内置的 RAG 实现 

- [检索型问答（Retrieval QA）](https://python.langchain.com.cn/docs/modules/chains/popular/vector_db_qa)

In [ ]:
!pip install langchain-chroma

In [ ]:
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import Docx2txtLoader
from langchain.prompts import PromptTemplate

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

# 加载文档
loader = Docx2txtLoader("体检中心问答.docx")
doc = loader.load()
content = doc[0].page_content

# 文档切分
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n\n"],
    chunk_size=0,
    chunk_overlap=0,
    length_function=len
)
docs = text_splitter.create_documents([content])

# 灌库
vectorstore = Chroma(
    collection_name="example_collection",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_langchain_db",  # 演示代码使用本地存储
)

# 演示需要，先重置向量数据库
vectorstore.reset_collection()
# 添加知识库
vectorstore.add_documents(documents=docs)

# LangChain内置的 RAG 实现
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    retriever=vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={'k': 2}
    ),
    return_source_documents=True  # 返回参考的知识
)

query = "空腹可以体检吗"
response = qa_chain.invoke(query)

print("======response=======")
print(response)

### 2.4、小结

1. 这部分能力 LangChain 的实现非常粗糙；
2. 实际生产中，建议自己实现，不建议用 LangChain 的工具。

## 三、记忆封装：Memory

### 3.1、对话上下文：ConversationBufferMemory

- 历史消息不限长度

In [1]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

history = ConversationBufferMemory()
# input和output配对使用
history.save_context({"input": "你好啊"}, {"output": "你也好啊"})

print(history.load_memory_variables({}))

history.save_context({"input": "你再好啊"}, {"output": "你又好啊"})

print(history.load_memory_variables({}))

{'history': 'Human: 你好啊\nAI: 你也好啊'}
{'history': 'Human: 你好啊\nAI: 你也好啊\nHuman: 你再好啊\nAI: 你又好啊'}


### 3.2、只保留固定长度窗口的上下文：ConversationBufferWindowMemory


In [2]:
from langchain.memory import ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(k=2)
window.save_context({"input": "第一轮问"}, {"output": "第一轮答"})
window.save_context({"input": "第二轮问"}, {"output": "第二轮答"})
window.save_context({"input": "第三轮问"}, {"output": "第三轮答"})
print(window.load_memory_variables({}))

{'history': 'Human: 第二轮问\nAI: 第二轮答\nHuman: 第三轮问\nAI: 第三轮答'}


### 3.3 根据 Token 数限定 Memory 大小 ConversationTokenBufferMemory

- https://python.langchain.com/docs/modules/memory/types/token_buffer

In [8]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

memory = ConversationTokenBufferMemory(
    llm = ChatOpenAI(),
    max_token_limit=20
)

memory.save_context({"input": "你好啊"}, {"output": "你好，我是你的AI助手"})
memory.save_context({"input": "你会干什么"}, {"output": "我什么都会"})

print(memory.load_memory_variables({}))

{'history': 'AI: 我什么都会'}


### 3.4、自动对历史信息做摘要：ConversationSummaryMemory


In [1]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    # buffer="The conversation is between a customer and a sales."
    buffer="以中文表示"
)
memory.save_context({"input": "你好"}, {"output": "你好，我是你的AI助手。我能为你回答有关AGIClass的各种问题。"})

print(memory.load_memory_variables({}))

{'history': '\n人类问AI对人工智能的看法。AI认为人工智能是一种积极的力量，因为它能帮助人类发挥他们的全部潜力。人类向AI打招呼，AI回应说它是人类的AI助手，可以回答关于AGIClass的各种问题。'}


### 3.5、更多类型
  
- VectorStoreRetrieverMemory: 将 Memory 存储在向量数据库中，根据用户输入检索回最相关的部分
  - https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory

### 3.6、小结

1. LangChain 的 Memory 管理机制属于可用的部分，尤其是简单情况如按轮数或按 Token 数管理；
2. 对于复杂情况，它不一定是最优的实现，例如检索向量库方式，建议根据实际情况和效果评估。

## 四、LangChain Expression Language (LCEL)

LangChain Expression Language（LCEL）是一种声明式语言，可轻松组合不同的调用顺序构成 Chain。

<img src="langchain.png" style="margin-left: 0px" width=600px>

LCEL的一些亮点包括：

1. **流支持**：使用 LCEL 构建 Chain 时，你可以获得最佳的首个令牌时间（即从输出开始到首批输出生成的时间）。对于某些 Chain，这意味着可以直接从LLM流式传输令牌到流输出解析器，从而以与 LLM 提供商输出原始令牌相同的速率获得解析后的、增量的输出。

2. **异步支持**：任何使用 LCEL 构建的链条都可以通过同步API（例如，在 Jupyter 笔记本中进行原型设计时）和异步 API（例如，在 LangServe 服务器中）调用。这使得相同的代码可用于原型设计和生产环境，具有出色的性能，并能够在同一服务器中处理多个并发请求。

3. **优化的并行执行**：当你的 LCEL 链条有可以并行执行的步骤时（例如，从多个检索器中获取文档），我们会自动执行，无论是在同步还是异步接口中，以实现最小的延迟。

4. **重试和回退**：为 LCEL 链的任何部分配置重试和回退。这是使链在规模上更可靠的绝佳方式。目前我们正在添加重试/回退的流媒体支持，因此你可以在不增加任何延迟成本的情况下获得增加的可靠性。

5. **访问中间结果**：对于更复杂的链条，访问在最终输出产生之前的中间步骤的结果通常非常有用。这可以用于让最终用户知道正在发生一些事情，甚至仅用于调试链条。你可以流式传输中间结果，并且在每个LangServe服务器上都可用。

6. **输入和输出模式**：输入和输出模式为每个 LCEL 链提供了从链的结构推断出的 Pydantic 和 JSONSchema 模式。这可以用于输入和输出的验证，是 LangServe 的一个组成部分。

7. **无缝LangSmith跟踪集成**：随着链条变得越来越复杂，理解每一步发生了什么变得越来越重要。通过 LCEL，所有步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。

8. **无缝LangServe部署集成**：任何使用 LCEL 创建的链都可以轻松地使用 LangServe 进行部署。

原文：https://python.langchain.com/v0.1/docs/expression_language/

### 看个例子

In [1]:
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

# 向量数据库
vectorstore = Chroma(
    collection_name="example_collection",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_langchain_db",  # 演示代码使用本地存储
)
# 演示需要，先重置向量数据库
vectorstore.reset_collection()

# 添加数据
vectorstore.add_texts(["Sam Altman是OpenAI的CEO", "Sam Altman被解雇了", "Sam Altman被复职了"])

# 检索接口
retriever = vectorstore.as_retriever()

# Prompt模板
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

# 模型
model = OpenAI()

# LCEL 表达式
retrieval_chain = (
        {"question": RunnablePassthrough(), "context": retriever}
        | prompt
        | model
        | StrOutputParser()
)

output = retrieval_chain.invoke("OpenAI的CEO是谁")
print(output)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3




Sam Altman


为什么使用LCEL？

https://python.langchain.com/v0.1/docs/expression_language/why/

### 通过 LCEL，还可以实现

1. 配置运行时变量：https://python.langchain.com/docs/expression_language/how_to/configure
2. 故障回退：https://python.langchain.com/docs/expression_language/how_to/fallbacks
3. 并行调用：https://python.langchain.com/docs/expression_language/how_to/map
4. 逻辑分支：https://python.langchain.com/docs/expression_language/how_to/routing
5. 调用自定义流式函数：https://python.langchain.com/docs/expression_language/how_to/generators
6. 链接外部Memory：https://python.langchain.com/docs/expression_language/how_to/message_history

更多例子：https://python.langchain.com/docs/expression_language/cookbook/

## 五、智能体架构：Agent


### 5.1 回忆：什么是智能体（Agent）

将大语言模型作为一个推理引擎。给定一个任务，智能体自动生成完成任务所需的步骤，执行相应动作（例如选择并调用工具），直到任务完成。

<img src="agent-overview.png" style="margin-left: 0px" width=800px>


Agent的类型有很多，详见文档：https://python.langchain.com/v0.1/docs/modules/agents/agent_types/ ， 下面介绍两种智能体：ReAct 和 Self Ask With Search

### 5.2 先定义一些工具：Tools

- 可以是一个函数或三方 API：https://python.langchain.com/v0.2/docs/integrations/tools/

- 也可以把一个 Chain 或者 Agent 的 run()作为一个 Tool

- 下面代码使用到SerpAPI搜索工具，使用前需要配置好`SERPAPI_API_KEY`去这里申请：https://serpapi.com/search-api


In [1]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool, tool

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

import calendar
import dateutil.parser as parser

# 使用SerpAPI搜索工具，搜索引擎替换为Baidu（默认是google）
# params = {
#   "engine": "Baidu"
# }
search = SerpAPIWrapper()

# 自定义工具
@tool("weekday")
def weekday(date_str: str) -> str:
    # llm会利用下面的注释来识别函数的功能
    """Convert date to weekday name"""
    d = parser.parse(date_str)
    return calendar.day_name[d.weekday()]

tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
    ),
    weekday
]

### 5.3 智能体类型：ReAct （Reasoning + Acting）

- LangChain文档：https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/

- ReAct论文：https://react-lm.github.io/ 

- 使用前先安装langchainhub， `!pip install langchainhub`。

- 需要配置好`LANGCHAIN_API_KEY` 或 `LANGCHAIN_HUB_API_KEY`，去这里申请：https://smith.langchain.com/hub


<img src="ReAct.png" style="margin-left: 0px" width=600px>


In [10]:
from langchain import hub
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor, create_react_agent

# 下载一个现有的prompt模板
prompt = hub.pull("hwchase17/react")
print("=========prompt模板===========")
print(prompt.template)

llm = OpenAI()
# 定义一个agent：需要大模型、工具集和prompt模板
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
# 定义一个执行器：需要agent对象和工具集，verbose=True 会打印中间过程
agent_exector = AgentExecutor(agent=agent, tools=tools, verbose=True)
# 执行
print("=========执行过程===========")
agent_exector.invoke({"input": "周杰伦生日那天是星期几"})

=========prompt模板===========
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
=========执行过程===========


> Entering new AgentExecutor chain...
 I need to know what day of the week it is on Jay Chou's birthday
Action: weekday
Action Input: '01/18/2021'
Monday I now know the day of the week Jay Chou's birthday is on
Final Answer: Monday

> Finished chain.


{'input': '周杰伦生日那天是星期几', 'output': 'Monday'}

### 5.4 智能体类型：Self-Ask With Search

- 这是一种自问自答形式的agent

- LangChain文档：https://python.langchain.com/v0.1/docs/modules/agents/agent_types/self_ask_with_search/

- self_ask_with_search_agent 只能传一个名为‘Intermediate Answer’的tool


In [3]:
from langchain_community.tools.tavily_search import TavilyAnswer
from langchain_community.utilities import SerpAPIWrapper


from langchain_openai import OpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent

llm = OpenAI(temperature=0)

# 使用SerpAPI搜索工具，搜索引擎替换为Baidu（默认是google）
params = {
  "engine": "Baidu"
}
search = SerpAPIWrapper(params=params)

# self_ask_with_search_agent 只能传一个名为‘Intermediate Answer’的tool
tools = [
    # TavilyAnswer(max_results=1, name="Intermediate Answer"),
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search.",
    )
]

prompt = hub.pull("hwchase17/self-ask-with-search")
print("=========prompt模板===========")
print(prompt.template)

llm = OpenAI()

agent = create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "吴京的老婆的主持过哪些节目"})

=========prompt模板===========
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was t

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  Yes.
Followup: 谁是吴京的老婆?


### 5.5 OpenAI Assistants

支持OpenAI的Assistants API

In [101]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "10减4的差的2.3次方是多少"})

print(output[0].content[0].text.value)

\(10\) 减去 \(4\) 的差的 \(2.3\) 次方是约 \(61.624\)。


<div class="alert alert-success">
<b>划重点：</b>
<ol>
<li>ReAct 是比较常用的 智能体</li>
<li>SelfAskWithSearch 更适合需要层层推理的场景（例如知识图谱）</li>
<li>OpenAI Assistants 不是万能的，后面课程中我们会专门讲 Agent 的实现</li>
</ol>
</div>

## 六、LangServe

LangServe 用于将 Chain 或者 Runnable 部署成一个 REST API 服务。

In [ ]:
# 安装 LangServe
!pip install "langserve[all]"

# 也可以只安装一端
# !pip install "langserve[client]"
# !pip install "langserve[server]"

### 6.1、Server端

```python
#!/usr/bin/env python
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langserve import add_routes
import uvicorn

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话")
add_routes(
    app,
    prompt | model,
    path="/joke",
)

if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8080)
```

### 6.2、Client端

```python
from langserve import RemoteRunnable

joke_chain = RemoteRunnable("http://localhost:8080/joke/")

joke_chain.invoke({"topic": "小明"})
```

## 总结

1. LangChain 随着版本迭代可用性有明显提升
2. 使用 LangChain 要避开存在大量代码内 Prompt 的模块
3. 它的内置基础工具，建议充分测试效果后再决定是否使用